In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#from src.DecisionTreeClassifier import DecisionTreeClassifier
from NJCleaner import NJCleaner

In [2]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain 
        
        # for leaf node
        self.value = value

In [20]:
import numpy as np

#from src.Node import Node

class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        
        self.root = None
        
        self.min_samples_split = min_samples_split 
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            best_split = self.get_best_split(dataset, num_samples, num_features)
            if best_split == {}:
                print("depth",curr_depth)
                print(dataset)

            if best_split["info_gain"]>0: 
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                return Node(best_split["feature_index"], best_split["threshold"],
                            left_subtree, right_subtree, best_split["info_gain"])
        

        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        best_split = {}
        max_info_gain = -float("inf")
        
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            for threshold in possible_thresholds: 
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
      
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        
        if tree.value!=None: 
            return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

In [4]:
#col_name = ['sepal_lenght', 'sepal_width', 'petal_length', 'petal_width', 'type']
#data = pd.read_csv("Iris.csv",skiprows=1, header=None, names=col_name)
#data = pd.read_csv("Iris.csv",names=col_name, header=0)
#print(data)

In [5]:
#X = data.iloc[:, :-1].values
#Y = data.iloc[:, -1].values.reshape(-1,1)
#X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=.2, random_state=41)


In [6]:
#hfdata = pd.read_csv("2018_03.csv")
#print(hfdata)
njc = NJCleaner("2018_03.csv")
hfdata = njc.prep_df()

In [7]:
X = hfdata.iloc[:,:-1].values
Y = hfdata.iloc[:,-1].values.reshape(-1,1)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=.2, random_state=41)

In [22]:
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=5)
classifier.fit(X_train, Y_train)

Y_pred = classifier.predict(X_test)
print(accuracy_score(Y_test, Y_pred))

0.7922067234497798


In [23]:
classifier.print_tree()

X_0 <= 2.0 ? 0.005681898245062045
 left:X_2 <= 145 ? 0.003781242518398703
  left:X_4 <= Northeast Corrdr ? 0.001443853314887264
    left:X_2 <= 104 ? 0.0026346769443702234
        left:X_0 <= 1.0 ? 0.002164511751077014
                left:X_4 <= No Jersey Coast ? 0.00011635207480281684
                                left:0
                                right:0
                right:X_4 <= Montclair-Boonton ? 0.0012021741335261071
                                left:0
                                right:0
        right:X_2 <= 113 ? 0.004387293594947772
                left:X_0 <= 1.0 ? 0.0034215719424994795
                                left:0
                                right:0
                right:X_4 <= Main Line ? 0.0037629434249306787
                                left:0
                                right:0
    right:X_2 <= 107 ? 0.00034835621215468376
        left:X_2 <= 105 ? 0.0012351638157652584
                left:X_1 <= 63 ? 0.000449396137192741
          

In [10]:
C = [1, 2, 3, 4, 5, 6]

for choice in C:
    

SyntaxError: incomplete input (3729092607.py, line 4)